<a href="https://colab.research.google.com/github/ccasanoval/RLtests/blob/master/DoomV4b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DoomV4b

RL  = Stable Baseline 3 : PPO

ENV = Gymnasium + VizDoom

URL = https://github.com/AKapich/Reinforcement_Learning_Doom

URL = https://github.com/AKapich/Reinforcement_Learning_Doom/blob/main/MyWayHome/my_way_home_env.py

URL = https://stable-baselines3.readthedocs.io/en/master/


###.
### ALSO TEST : RLlib

###.
### ALSTO TRY: callbacks

https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/sb3/4_callbacks_hyperparameter_tuning.ipynb#scrollTo=adsKMvDkRUn0

###.
### ALSO READ: Algorithm Distilation + In Context RL

https://www.youtube.com/watch?v=BkWLCrLapQo

https://github.com/licong-lin/in-context-rl
    
###.
### ALSO READ:

Automatic hyperparameter optimization

To optimize the parameters of C, we chose the Covariance-Matrix Adaptation Evolution Strategy (CMA-ES)

evolution strategies (ES)

###.
Do androids dream of electric sheep?

https://worldmodels.github.io/

https://github.com/hardmaru/WorldModelsExperiments



In [6]:
!pip install vizdoom
!pip install stable_baselines3

In [7]:
modelType = "PPO"     # @param {type:"string"}
modelName = "DoomHome"+modelType # @param {type:"string"}
modelNew = True       # @param {type:"boolean"}
modelTrain = True     # @param {type:"boolean"}

print(modelName)

DoomHomePPO


In [14]:
############### GYM ENV == VIZ DOOM ###########################################
from vizdoom import DoomGame, GameVariable
import numpy as np
from gymnasium import Env
from gymnasium.spaces import Discrete, Box
import cv2

class MyWayHomeGym(Env):
    def __init__(self, scenario, render=True, number_of_actions=3):
        self.game = DoomGame()
        self.game.load_config(f"{scenario}.cfg")

        # self.game.set_mode(Mode.SPECTATOR)  # spectator

        self.game.add_available_game_variable(GameVariable.POSITION_X)
        self.game.add_available_game_variable(GameVariable.POSITION_Y)
        self.game.add_available_game_variable(GameVariable.POSITION_Z)

        self.pos = None

        self.game.set_window_visible(render)
        self.game.init()

        self.observation_space = Box(
            low=0, high=255, shape=(100, 160, 1), dtype=np.uint8
        )
        self.number_of_actions = number_of_actions
        self.action_space = Discrete(number_of_actions)
        self.wins = 0


    def step(self, action):

        actions = np.identity(self.number_of_actions)
        # tics need to be more than 1 to learn (tics = 4)
        tics = 1
        if modelTrain: tics = 4
        reward = self.game.make_action(actions[action], tics = tics)
        if modelTrain:
            reward *= 100
        #    if reward < 0: reward *= 10

        dist = 0
        movement_reward = 0
        if self.game.get_state():

            # MOVEMENT REWARD
            if modelTrain:
                _, pos_x, pos_y, pos_z = self.game.get_state().game_variables
                pos = np.array([pos_x, pos_y, pos_z])
                if self.pos is not None:
                    dist = np.sqrt(np.sum((pos - self.pos) ** 2))
                    movement_reward = dist * 0.005
                    self.pos = pos

            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)
            info = self.game.get_state().game_variables[0]  # ammo
        else:
            state = np.zeros(self.observation_space.shape)
            info = 0

        info = {"info": info}
        terminated = self.game.is_episode_finished()

        truncated = (
            self.game.is_player_dead()
            or self.game.is_player_dead()
            or self.game.is_player_dead()
        )

        if reward > 0:
            self.wins = self.wins + 1
            print(f"-************win reward = {reward} wins = {self.wins} *************************")
        elif terminated: print(f"-------------end reward = {reward} / sum = {reward+movement_reward} dist = {dist} movement_reward = {movement_reward} ")
        #elif modelTrain: print(f"-------------end reward = {reward} / sum = {reward+movement_reward} dist = {dist} movement_reward = {movement_reward} ")

        if modelTrain: reward += movement_reward
        return state, reward, terminated, truncated, info


    def reset(self, seed=0):
        self.game.new_episode()
        state = self.game.get_state().screen_buffer

        if self.game.get_state():
            info = self.game.get_state().game_variables[0]  # ammo
        else:
            info = 0

        return (self.grayscale(state), {"ammo": info})

    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (160, 100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100, 160, 1))
        return state

    def close(self):
        self.game.close()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:

##################### SB3 : CALLBACK ##########################################
from stable_baselines3.common.callbacks import BaseCallback
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, verbose=1, name="?"):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.name = name

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = '{}_{}'.format(self.name, self.n_calls)
            self.model.save(model_path)
        return True



In [ ]:

###################### TRAIN == SB3 ###########################################
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO


callback = TrainAndLoggingCallback(check_freq=10000, name=modelName)
env = MyWayHomeGym(render=False, scenario="my_way_home")

if modelNew:
    model = PPO(
        "CnnPolicy",
        env,
        verbose=1,
        seed=0,
        learning_rate=0.0001,
        n_steps=2048,
    )
else:
    model = PPO.load(modelName, env=env)

# TRAIN
if modelTrain:
    model.learn(
        total_timesteps=200000,
        callback=callback,
    )



Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
-------------end reward = -0.04 / sum = -0.04 dist = 0 movement_reward = 0 
-------------end reward = -0.04 / sum = -0.04 dist = 0 movement_reward = 0 
-------------end reward = -0.04 / sum = -0.04 dist = 0 movement_reward = 0 
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 525      |
|    ep_rew_mean     | -21      |
| time/              |          |
|    fps             | 240      |
|    iterations      | 1        |
|    time_elapsed    | 8        |
|    total_timesteps | 2048     |
---------------------------------
-------------end reward = -0.04 / sum = -0.04 dist = 0 movement_reward = 0 
-------------end reward = -0.04 / sum = -0.04 dist = 0 movement_reward = 0 
-************win reward = 99.96000000000001 wins = 1 *************************
-------------end reward = -0.01 / sum = -0.01 dist = 0 movement_re

In [ ]:
###################### TEST == SB3 ########################################
model = PPO.load(modelName, env=env)
env = env = MyWayHomeGym(render=True, scenario="my_way_home")
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
env.close()

print(f"mean_reward:{mean_reward:.2f}")
print(f"std_reward:{std_reward:.2f}")
